In [2]:
import os
import pandas as pd
import numpy as np
from future_and_exchange import futures_exchange_dict, futures_english_dict2


In [8]:
directory = '结果文件/合并结果/'


In [7]:
def add_score_column(file_path):  # 其实这个程序里的encoding根本没用，出现乱码是之前中间写得有问题
    file_name = file_path.split('/')[-1]  # Get just the filename from path
    keywords_part = file_name.split('_')[0]
    keywords = keywords_part.split(' ')

    df = pd.read_csv(file_path, encoding='utf-8-sig')  # 生成score用
    conditions = [
        (df['Positive'] > df['Neutral']) & (df['Positive'] > df['Negative']),
        (df['Negative'] > df['Neutral']) & (df['Negative'] > df['Positive']),
        (df['Neutral'] > df['Positive']) & (df['Neutral'] > df['Negative'])
    ]
    choices = [1, -1, 0]
    df['score'] = np.select(conditions, choices, default=np.nan)
    df['time'] = df['发布时间'].apply(parse_date).dt.date
    def check_keywords(text):
        if not isinstance(text, str):
            return 0
        has_keyword0 = any(keyword in text for keyword in [keywords[0]]) if len(keywords) > 0 else False
        has_keyword1 = any(keyword in text for keyword in [keywords[1]]) if len(keywords) > 1 else False

        if has_keyword0 and has_keyword1:  # 同时2个关键词为3
            return 3
        elif has_keyword0:  # 只有期货名的为1
            return 1
        elif has_keyword1:  # 只有'期货'两个字的为2
            return 2
        else:
            return 0  # 毛都没有的为0

    df['filter_result'] = df['微博正文'].apply(check_keywords)

    df.to_csv(file_path, index=False, encoding='utf-8-sig')

In [5]:
def parse_date(s):
    if '-' in s:
        # print(pd.to_datetime(s, format="%Y-%m-%d %H:%M"))
        return pd.to_datetime(s, format="%Y-%m-%d %H:%M")
    else:
        # print(pd.to_datetime(s, format="%Y/%m/%d %H:%M"))
        return pd.to_datetime(s, format="%Y/%m/%d %H:%M")

In [9]:
for file in os.listdir(directory):
    if file.endswith('.csv'):
        file_path = os.path.join(directory, file)
        add_score_column(file_path)

C:\Users\akane\AppData\Local\Temp\ipykernel_37952\469494641.py:6: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, encoding='utf-8-sig')  # 生成score用
C:\Users\akane\AppData\Local\Temp\ipykernel_37952\469494641.py:6: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, encoding='utf-8-sig')  # 生成score用
C:\Users\akane\AppData\Local\Temp\ipykernel_37952\469494641.py:6: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, encoding='utf-8-sig')  # 生成score用
C:\Users\akane\AppData\Local\Temp\ipykernel_37952\469494641.py:6: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, encoding='utf-8-sig')  # 生成score用
C:\Users\akane\AppData\Local\Temp\ipykernel_37952\469494641.py:6: DtypeWarni

In [9]:
directory2 = '结果文件/去重合并结果/'


In [10]:
for file in os.listdir(directory2):
    if file.endswith('.csv'):
        file_path = os.path.join(directory2, file)
        add_score_column(file_path)

C:\Users\akane\AppData\Local\Temp\ipykernel_8684\4275787505.py:6: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, encoding='utf-8-sig')  # 生成score用
C:\Users\akane\AppData\Local\Temp\ipykernel_8684\4275787505.py:6: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, encoding='utf-8-sig')  # 生成score用
C:\Users\akane\AppData\Local\Temp\ipykernel_8684\4275787505.py:6: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, encoding='utf-8-sig')  # 生成score用
C:\Users\akane\AppData\Local\Temp\ipykernel_8684\4275787505.py:6: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, encoding='utf-8-sig')  # 生成score用
C:\Users\akane\AppData\Local\Temp\ipykernel_8684\4275787505.py:6: DtypeWarni

In [3]:
def load_all_data(directory):
    data_dict = {}
    for file in os.listdir(directory):
        if file.endswith('.csv'):
            keyword = file.split('_')[0]
            file_path = os.path.join(directory, file)
            df = pd.read_csv(file_path)
            data_dict[keyword] = df
    return data_dict

In [4]:
def summarize_column_describe(data_dict, column_name):
    result_df = pd.DataFrame()
    for keyword, df in data_dict.items():
        if column_name in df.columns:
            describe_series = df[column_name].describe()
            describe_series.name = keyword
            result_df = pd.concat([result_df, describe_series], axis=1)
    result_df = result_df.T
    return result_df

In [5]:
def summarize_column_describe2(data_dict, column_name):
    result_df = pd.DataFrame()
    for keyword, df in data_dict.items():
        if column_name in df.columns:
            describe_series = df[column_name].describe()
            describe_series.name = keyword
            result_df = pd.concat([result_df, describe_series], axis=1)

    result_df = result_df.T

    # 提取关键字的前缀
    result_df['english_name'] = result_df.index.str.split(' ').str[0].map(futures_english_dict2).fillna(None)
    result_df['exchange'] = result_df.index.str.split(' ').str[0].map(futures_exchange_dict).fillna(None)

    return result_df

In [6]:
def summarize_column_describe3(data_dict, column_name):
    result_df = pd.DataFrame()
    for keyword, df in data_dict.items():
        if column_name in df.columns:
            describe_series = df[column_name].describe()
            describe_series.name = keyword
            total_sum = df[column_name].sum()
            describe_series['total'] = total_sum
            result_df = pd.concat([result_df, describe_series], axis=1)

    result_df = result_df.T

    # 提取关键字的前缀
    result_df['english_name'] = result_df.index.str.split(' ').str[0].map(futures_english_dict2).fillna(None)
    result_df['exchange'] = result_df.index.str.split(' ').str[0].map(futures_exchange_dict).fillna(None)

    return result_df

In [7]:
def summarize_column_describe4(data_dict, column_name):
    result_df = pd.DataFrame()
    for keyword, df in data_dict.items():
        if column_name in df.columns:
            df['time'] = pd.to_datetime(df[column_name]).dt.date
            describe_series = df['time'].describe()
            describe_series.name = keyword
            result_df = pd.concat([result_df, describe_series], axis=1)

    result_df = result_df.T

    # 提取关键字的前缀
    result_df['english_name'] = result_df.index.str.split(' ').str[0].map(futures_english_dict2).fillna(None)
    result_df['exchange'] = result_df.index.str.split(' ').str[0].map(futures_exchange_dict).fillna(None)

    return result_df

In [8]:
def summarize_column_describe5(data_dict, column_name):
    result_df = pd.DataFrame()
    for keyword, df in data_dict.items():
        if column_name in df.columns:
            df[column_name]= df[column_name].astype(str)
            describe_series = df[column_name].describe()
            describe_series.name = keyword
            result_df = pd.concat([result_df, describe_series], axis=1)

    result_df = result_df.T

    # 提取关键字的前缀
    result_df['english_name'] = result_df.index.str.split(' ').str[0].map(futures_english_dict2).fillna(None)
    result_df['exchange'] = result_df.index.str.split(' ').str[0].map(futures_exchange_dict).fillna(None)

    return result_df

In [9]:
def summarize_column_describe6(data_dict, column_name):
    result_df = pd.DataFrame()
    for keyword, df in data_dict.items():
        if column_name in df.columns:
            groupby_df = df.groupby('time')[column_name].mean()
            describe_series = groupby_df.describe()
            describe_series.name = keyword
            result_df = pd.concat([result_df, describe_series], axis=1)

    result_df = result_df.T

    # 提取关键字的前缀
    result_df['english_name'] = result_df.index.str.split(' ').str[0].map(futures_english_dict2).fillna(None)
    result_df['exchange'] = result_df.index.str.split(' ').str[0].map(futures_exchange_dict).fillna(None)

    return result_df

In [10]:
data_dict = load_all_data(directory2)


C:\Users\akane\AppData\Local\Temp\ipykernel_34584\2832148986.py:7: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
C:\Users\akane\AppData\Local\Temp\ipykernel_34584\2832148986.py:7: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
C:\Users\akane\AppData\Local\Temp\ipykernel_34584\2832148986.py:7: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
C:\Users\akane\AppData\Local\Temp\ipykernel_34584\2832148986.py:7: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
C:\Users\akane\AppData\Local\Temp\ipykernel_34584\2832148986.py:7: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
C:\Users\a

In [11]:
summarize_column_describe3(data_dict, '转发数')

,count,mean,std,min,25%,50%,75%,max,total,english_name,exchange
20号胶 期货,6656.0,0.349008,2.569259,0.0,0.0,0.0,0.0,109.0,2323.0,Rubber 20,上海期货交易所
PTA 期货,52573.0,0.923820,10.184961,0.0,0.0,0.0,0.0,1539.0,48568.0,PTA,郑州商品交易所
丁二烯橡胶 期货,320.0,0.818750,1.994890,0.0,0.0,0.0,1.0,15.0,262.0,Butadiene Rubber,上海期货交易所
不锈钢 期货,17775.0,0.392574,8.371281,0.0,0.0,0.0,0.0,1001.0,6978.0,Stainless Steel,上海期货交易所
乙二醇 期货,26957.0,0.498052,5.008670,0.0,0.0,0.0,0.0,543.0,13426.0,Ethylene Glycol,大连商品交易所
...,...,...,...,...,...,...,...,...,...,...,...
锰硅 期货,11235.0,0.501291,4.221088,0.0,0.0,0.0,0.0,321.0,5632.0,Silicon Manganese,郑州商品交易所
镍 期货,67420.0,0.704628,8.232958,0.0,0.0,0.0,0.0,1032.0,47506.0,Nickel,上海期货交易所
阴极铜 期货,9313.0,0.953506,11.049710,0.0,0.0,0.0,0.0,666.0,8880.0,Cathode Copper,上海期货交易所
鸡蛋 期货,34790.0,1.101638,38.314317,0.0,0.0,0.0,0.0,6910.0,38326.0,Eggs,大连商品交易所


In [7]:
summarize_column_describe(data_dict, '评论数')

,count,mean,std,min,25%,50%,75%,max
20号胶 期货,6656.0,0.667518,7.276272,0.0,0.0,0.0,0.0,503.0
PTA 期货,52573.0,1.124684,7.172007,0.0,0.0,0.0,0.0,670.0
丁二烯橡胶 期货,320.0,0.756250,2.024139,0.0,0.0,0.0,0.0,10.0
不锈钢 期货,17775.0,0.566245,5.140218,0.0,0.0,0.0,0.0,323.0
乙二醇 期货,24236.0,0.786846,5.591097,0.0,0.0,0.0,0.0,506.0
...,...,...,...,...,...,...,...,...
锰硅 期货,11235.0,0.660703,4.216825,0.0,0.0,0.0,0.0,171.0
镍 期货,67420.0,0.919608,8.502657,0.0,0.0,0.0,0.0,1358.0
阴极铜 期货,303.0,0.531353,3.063053,0.0,0.0,0.0,0.0,42.0
鸡蛋 期货,34790.0,1.303765,16.417026,0.0,0.0,0.0,0.0,2757.0


In [8]:
summarize_column_describe(data_dict, '点赞数')

,count,mean,std,min,25%,50%,75%,max
20号胶 期货,6656.0,2.125451,12.803516,0.0,0.0,0.0,1.0,459.0
PTA 期货,52573.0,3.273696,27.181404,0.0,0.0,0.0,1.0,4703.0
丁二烯橡胶 期货,320.0,3.150000,8.916330,0.0,0.0,0.0,2.0,110.0
不锈钢 期货,17775.0,2.317975,21.864826,0.0,0.0,0.0,0.0,1591.0
乙二醇 期货,24236.0,2.252352,12.635221,0.0,0.0,0.0,1.0,602.0
...,...,...,...,...,...,...,...,...
锰硅 期货,11235.0,1.846195,10.800917,0.0,0.0,0.0,1.0,618.0
镍 期货,67420.0,3.859478,30.957974,0.0,0.0,0.0,1.0,2574.0
阴极铜 期货,303.0,2.874587,19.703870,0.0,0.0,0.0,1.0,292.0
鸡蛋 期货,34790.0,3.686002,93.758520,0.0,0.0,0.0,1.0,11762.0


In [17]:
summarize_column_describe4(data_dict, '发布时间')

,count,unique,top,freq,english_name,exchange
20号胶 期货,6656,1065,2019-08-12,165,Rubber 20,上海期货交易所
PTA 期货,52573,3685,2023-12-18,190,PTA,郑州商品交易所
丁二烯橡胶 期货,320,120,2023-07-28,18,Butadiene Rubber,上海期货交易所
不锈钢 期货,17775,1817,2019-09-25,154,Stainless Steel,上海期货交易所
乙二醇 期货,26957,1854,2021-10-20,119,Ethylene Glycol,大连商品交易所
...,...,...,...,...,...,...
锰硅 期货,11235,1798,2021-09-23,118,Silicon Manganese,郑州商品交易所
镍 期货,67420,3154,2022-03-08,1297,Nickel,上海期货交易所
阴极铜 期货,9313,2284,2020-11-19,138,Cathode Copper,上海期货交易所
鸡蛋 期货,34790,3359,2020-12-14,132,Eggs,大连商品交易所


In [14]:
summarize_column_describe(data_dict, 'ip')

,count,unique,top,freq
20号胶 期货,1739,30,湖北,492
PTA 期货,3732,35,湖北,1132
丁二烯橡胶 期货,213,17,湖北,46
不锈钢 期货,3581,33,湖北,984
乙二醇 期货,5537,31,湖北,3296
...,...,...,...,...
锰硅 期货,1138,25,湖北,325
镍 期货,10308,40,湖北,3944
阴极铜 期货,1729,39,湖北,275
鸡蛋 期货,2477,37,湖北,849


In [18]:
summarize_column_describe(data_dict, '用户昵称')

,count,unique,top,freq
20号胶 期货,6656,1415,银河期货,300
PTA 期货,52573,5699,北斗言期,1451
丁二烯橡胶 期货,320,129,上海证券报,40
不锈钢 期货,17775,2196,中国钢铁现货网,4128
乙二醇 期货,26957,2435,期市早班车,1016
...,...,...,...,...
锰硅 期货,11235,1967,鲲鹏DIY小牛,260
镍 期货,67420,7177,北斗言期,1468
阴极铜 期货,9313,2614,中国国际期货清远营业部,713
鸡蛋 期货,34790,4673,芝华数据,1275


In [21]:
summarize_column_describe5(data_dict, 'user_id')

,count,unique,top,freq,english_name,exchange
20号胶 期货,6656,1415,2988354305,300,Rubber 20,上海期货交易所
PTA 期货,52573,5698,5426606030,1451,PTA,郑州商品交易所
丁二烯橡胶 期货,320,129,1905628462,40,Butadiene Rubber,上海期货交易所
不锈钢 期货,17775,2196,2280596260,4128,Stainless Steel,上海期货交易所
乙二醇 期货,26957,2427,5812127807,1016,Ethylene Glycol,大连商品交易所
...,...,...,...,...,...,...
锰硅 期货,11235,1967,1651756987,260,Silicon Manganese,郑州商品交易所
镍 期货,67420,7177,5426606030,1468,Nickel,上海期货交易所
阴极铜 期货,9313,2614,2665926601,713,Cathode Copper,上海期货交易所
鸡蛋 期货,34790,4673,2885299602,1275,Eggs,大连商品交易所


In [11]:
summarize_column_describe(data_dict, 'user_authentication')

,count,unique,top,freq
20号胶 期货,6656,5,普通用户,3406
PTA 期货,52573,5,普通用户,27946
丁二烯橡胶 期货,320,4,普通用户,141
不锈钢 期货,17775,5,普通用户,11295
乙二醇 期货,24236,5,普通用户,13208
...,...,...,...,...
锰硅 期货,11235,5,普通用户,6469
镍 期货,67420,5,普通用户,37249
阴极铜 期货,303,5,普通用户,178
鸡蛋 期货,34790,5,普通用户,22291


In [12]:
summarize_column_describe(data_dict, 'valid')

,count,unique,top,freq
20号胶 期货,6656,2,True,5891
PTA 期货,52573,2,True,48873
丁二烯橡胶 期货,320,2,True,263
不锈钢 期货,17775,2,True,15675
乙二醇 期货,24236,2,True,22209
...,...,...,...,...
锰硅 期货,11235,2,True,10150
镍 期货,67420,2,True,59676
阴极铜 期货,303,2,True,229
鸡蛋 期货,34790,2,True,31958


In [37]:
summarize_column_describe6(data_dict, 'Neutral')

,count,mean,std,min,25%,50%,75%,max,english_name,exchange
20号胶 期货,990.0,0.407052,0.367242,0.000578,0.057511,0.290778,0.748641,0.997616,Rubber 20,上海期货交易所
PTA 期货,3659.0,0.548354,0.280887,0.000568,0.365581,0.552269,0.741250,0.997827,PTA,郑州商品交易所
丁二烯橡胶 期货,95.0,0.311323,0.360148,0.000674,0.008026,0.126152,0.585036,0.996485,Butadiene Rubber,上海期货交易所
不锈钢 期货,1718.0,0.471979,0.349405,0.000441,0.158720,0.429682,0.794562,0.998237,Stainless Steel,上海期货交易所
乙二醇 期货,1804.0,0.541004,0.296079,0.000526,0.360331,0.559479,0.746529,0.997829,Ethylene Glycol,大连商品交易所
...,...,...,...,...,...,...,...,...,...,...
锰硅 期货,1719.0,0.477212,0.347398,0.000439,0.181044,0.426210,0.798985,0.998100,Silicon Manganese,郑州商品交易所
镍 期货,3109.0,0.503915,0.300141,0.000395,0.301241,0.513536,0.706027,0.998164,Nickel,上海期货交易所
阴极铜 期货,2050.0,0.510265,0.410008,0.000425,0.048476,0.499739,0.992952,0.997994,Cathode Copper,上海期货交易所
鸡蛋 期货,3313.0,0.592784,0.291173,0.000556,0.405769,0.613719,0.822298,0.997762,Eggs,大连商品交易所


In [14]:
summarize_column_describe(data_dict, 'Positive')

,count,mean,std,min,25%,50%,75%,max
20号胶 期货,5891.0,0.345604,0.437795,0.000516,0.003613,0.024614,0.956255,0.999208
PTA 期货,48873.0,0.271743,0.410032,0.000406,0.002906,0.008247,0.676463,0.999203
丁二烯橡胶 期货,263.0,0.268721,0.404144,0.000643,0.004098,0.031099,0.393865,0.999064
不锈钢 期货,15675.0,0.270814,0.413328,0.000458,0.002433,0.006176,0.684986,0.999212
乙二醇 期货,22209.0,0.276799,0.407545,0.000435,0.003369,0.010867,0.677083,0.999262
...,...,...,...,...,...,...,...,...
锰硅 期货,10150.0,0.368384,0.446668,0.000440,0.003352,0.023566,0.968889,0.999144
镍 期货,59676.0,0.309920,0.425355,0.000438,0.003529,0.014755,0.882346,0.999251
阴极铜 期货,229.0,0.292138,0.412939,0.000484,0.002355,0.009626,0.766112,0.999255
鸡蛋 期货,31958.0,0.251000,0.397076,0.000427,0.002934,0.008680,0.445076,0.999199


In [15]:
summarize_column_describe(data_dict, 'Negative')

,count,mean,std,min,25%,50%,75%,max
20号胶 期货,5891.0,0.257527,0.394351,0.000240,0.000892,0.002905,0.586028,0.998801
PTA 期货,48873.0,0.174653,0.350324,0.000216,0.000841,0.001664,0.029128,0.999027
丁二烯橡胶 期货,263.0,0.385778,0.416722,0.000316,0.000923,0.193650,0.892389,0.998318
不锈钢 期货,15675.0,0.228166,0.384803,0.000246,0.001031,0.002226,0.293380,0.998962
乙二醇 期货,22209.0,0.187908,0.359618,0.000228,0.000834,0.001812,0.055813,0.999063
...,...,...,...,...,...,...,...,...
锰硅 期货,10150.0,0.242297,0.394648,0.000240,0.000824,0.002148,0.447980,0.999003
镍 期货,59676.0,0.178837,0.349325,0.000229,0.000844,0.001965,0.045525,0.998983
阴极铜 期货,229.0,0.116895,0.284087,0.000312,0.000709,0.001280,0.006544,0.998599
鸡蛋 期货,31958.0,0.160777,0.335294,0.000220,0.000950,0.001937,0.023772,0.999009


In [12]:
for_list = ['转发数', '评论数', '点赞数','user_id', '用户昵称', '发布时间', 'ip', 'user_authentication', 'valid', 'Neutral', 'Positive', 'Negative', 'score']
all_dfs = []
with pd.ExcelWriter('output.xlsx') as writer:
    for item in for_list:
        if item in ['转发数', '评论数', '点赞数',]:
            dfss = summarize_column_describe3(data_dict, item)
        elif item in ['user_id']:
            dfss = summarize_column_describe5(data_dict, item)
        elif item in ['Neutral', 'Positive', 'Negative', 'score']:
            dfss = summarize_column_describe6(data_dict, item)
        elif item in ['发布时间']:
            dfss = summarize_column_describe4(data_dict, item)
        else:
            dfss = summarize_column_describe(data_dict, item)
        dfss.to_excel(writer, sheet_name=item)
        dfss.columns = pd.MultiIndex.from_product([[item], dfss.columns])
        all_dfs.append(dfss)
    combined_df = pd.concat(all_dfs, axis=1)
    combined_df.to_excel(writer, sheet_name='All_Data')